In [1]:
import numpy as np
from scipy import optimize as opt

In [2]:
# Model Primitives
np.random.seed(1234567890)

nObs = 1000
beta = np.array([0.5, 0.5] , dtype=float)
income= np.random.uniform(size = nObs) # draws from standard normal
explVar = np.vstack([np.ones(nObs), income]).T

In [3]:
def simulateBinaryLogit(x, beta):
    nObs     = x.shape[0]
    nChoice  = 2;
    
    epsilon = np.random.gumbel(size = [nObs, nChoice])
    beta_augmented = np.vstack([beta, np.zeros(beta.shape)])
    
    utility = x @ beta_augmented + epsilon
    
    return np.argmax(utility, axis=1)

In [4]:
choice = simulateBinaryLogit(explVar, beta)

In [5]:
data = np.hstack((choice.reshape(nObs, 1), explVar))
data.shape

(1000, 3)

In [6]:
def calcLambda(x, beta):
    prob = np.exp(x @ beta)  / (1 + np.exp(x @ beta))
    return prob

In [7]:
def logLike_binaryLogit(beta, y, x):
    choiceProb   = calcLambda(x, beta);
    
    ll_i         = y * np.log(choiceProb) + (1 - y) * np.log(1 - choiceProb);
    logLike      = -(ll_i.sum())
    return -((-np.log(1 + np.exp(np.dot(x, beta)))).sum() + (y*(np.dot(x, beta))).sum())
    #return logLike

In [8]:
beta0 = np.ones(2)
opt.minimize(logLike_binaryLogit, beta0, args=(data[:,0], data[:,1:]) , method='BFGS', tol=1e-12)

      fun: 692.8698580664086
 hess_inv: array([[ 0.01668105, -0.0250069 ],
       [-0.0250069 ,  0.04933067]])
      jac: array([ 0.,  0.])
  message: 'Optimization terminated successfully.'
     nfev: 44
      nit: 6
     njev: 11
   status: 0
  success: True
        x: array([ 0.03956486, -0.13323202])